# 06: Orchestration - Putting It All Together

**Duration:** 1 hour

**What You'll Learn:**
- Sequential agent orchestration patterns
- Conditional workflow logic
- Error handling and retries
- State management across agents
- Performance monitoring

**What We're Building:**
An orchestrator that coordinates our three agents (Filter, Rating, Generator) into a complete procurement intelligence pipeline.

**The Challenge:**
Individual agents are simple. Coordinating them reliably is where real engineering happens.

---

## Orchestration Patterns

We'll use **sequential orchestration** with **conditional branching**:

```
Input: Tender
    ↓
[Filter Agent]
    ↓
Is Relevant? → No → END (filtered out)
    ↓ Yes
[Rating Agent]
    ↓
Score >= Threshold? → No → END (rated low)
    ↓ Yes
[Document Generator]
    ↓
Output: Complete Analysis + Bid Document
```

This is simple but effective. Later we'll compare with graph-based orchestration.

## Step 1: Setup and Import Agents

In [ ]:
!pip install httpx pydantic

In [ ]:
import httpx
import json
import asyncio
from datetime import datetime
from typing import List, Type, TypeVar, Optional
from pydantic import BaseModel, Field
from enum import Enum

BASE_URL = "http://localhost:1234/v1"
MODEL = "local-model"
T = TypeVar('T', bound=BaseModel)

print("✓ Imports ready")

## Step 2: Define Complete Data Models

We need models for all stages of the pipeline.

In [ ]:
# Input
class Tender(BaseModel):
    id: str
    title: str
    description: str
    organization: str
    deadline: str
    estimated_value: str | None = None

# Agent outputs
class TenderCategory(str, Enum):
    CYBERSECURITY = "cybersecurity"
    AI = "ai"
    SOFTWARE = "software"
    OTHER = "other"

class FilterResult(BaseModel):
    is_relevant: bool
    confidence: float
    categories: List[TenderCategory]
    reasoning: str

class RatingResult(BaseModel):
    overall_score: float
    strategic_fit: float
    win_probability: float
    effort_required: float
    strengths: List[str]
    risks: List[str]
    recommendation: str

class BidDocument(BaseModel):
    executive_summary: str
    technical_approach: str
    value_proposition: str
    timeline_estimate: str

# Pipeline result
class ProcessedTender(BaseModel):
    """Complete result from pipeline"""
    tender: Tender
    filter_result: Optional[FilterResult] = None
    rating_result: Optional[RatingResult] = None
    bid_document: Optional[BidDocument] = None
    status: str = "pending"  # pending, filtered_out, rated_low, complete, error
    processing_time: float = 0.0
    error_message: Optional[str] = None

print("✓ Models defined")

## Step 3: Build Agent Functions

Recreate the agents from previous notebooks in compact form.

In [ ]:
def build_structured_prompt(prompt: str, model_class: Type[BaseModel]) -> str:
    schema = model_class.model_json_schema()
    return f"""{prompt}\n\nCRITICAL: Respond with ONLY valid JSON matching this schema:\n{json.dumps(schema, indent=2)}\n\nReturn ONLY the raw JSON object."""

async def call_llm(
    prompt: str,
    response_model: Type[T],
    system_prompt: str,
    temperature: float = 0.1
) -> T:
    full_prompt = build_structured_prompt(prompt, response_model)
    
    async with httpx.AsyncClient(timeout=90.0) as client:
        response = await client.post(
            f"{BASE_URL}/chat/completions",
            json={
                "model": MODEL,
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": full_prompt}
                ],
                "temperature": temperature,
            },
        )
        
        result = response.json()
        content = result["choices"][0]["message"]["content"].strip()
        
        # Clean
        for marker in ["```json", "```"]:
            if content.startswith(marker):
                content = content[len(marker):]
            if content.endswith(marker):
                content = content[:-len(marker)]
        content = content.strip()
        
        data = json.loads(content)
        return response_model.model_validate(data)

print("✓ LLM helper ready")

In [ ]:
# Agent 1: Filter
async def filter_agent(tender: Tender) -> FilterResult:
    prompt = f"""Analyze this tender:\n\nTITLE: {tender.title}\nDESCRIPTION: {tender.description}\nORGANIZATION: {tender.organization}\n\nCRITERIA: Relevant if involves cybersecurity, AI/ML, or software development. NOT relevant if only hardware, infrastructure, or non-tech services."""
    
    system = "You are an expert procurement analyst. Be precise and conservative."
    
    return await call_llm(prompt, FilterResult, system, temperature=0.1)

# Agent 2: Rating
async def rating_agent(tender: Tender, categories: List[str]) -> RatingResult:
    prompt = f"""Rate this tender for a small tech consultancy:\n\nTENDER: {tender.title}\nCLIENT: {tender.organization}\nVALUE: {tender.estimated_value or 'Not specified'}\nCATEGORIES: {', '.join(categories)}\nDESCRIPTION: {tender.description}\n\nEvaluate on: Strategic Fit, Win Probability, Effort Required. Provide realistic scores (0-10)."""
    
    system = "You are a business development expert. Be analytical and realistic, not optimistic."
    
    return await call_llm(prompt, RatingResult, system, temperature=0.1)

# Agent 3: Generator
async def generator_agent(tender: Tender, categories: List[str], strengths: List[str]) -> BidDocument:
    prompt = f"""Create bid document for:\n\nTENDER: {tender.title}\nCLIENT: {tender.organization}\nEXPERTISE: {', '.join(categories)}\nSTRENGTHS: {', '.join(strengths)}\nREQUIREMENTS: {tender.description}\n\nGenerate professional content: Executive Summary, Technical Approach, Value Proposition, Timeline."""
    
    system = "You are an expert proposal writer with 15 years experience. Write persuasively but authentically."
    
    return await call_llm(prompt, BidDocument, system, temperature=0.7)

print("✓ Agents ready")

## Step 4: Build the Orchestrator

Now the key piece: coordinate the agents with conditional logic.

In [ ]:
class ProcurementOrchestrator:
    """
    Orchestrates the procurement intelligence pipeline
    
    Pipeline stages:
    1. Filter for relevance
    2. Rate opportunity (if relevant)
    3. Generate documents (if high-rated)
    
    Includes error handling, conditional logic, and telemetry.
    """
    
    def __init__(
        self,
        min_confidence: float = 0.6,
        min_score: float = 7.0
    ):
        self.min_confidence = min_confidence
        self.min_score = min_score
    
    async def process_tender(self, tender: Tender) -> ProcessedTender:
        """Process a tender through the complete pipeline"""
        
        start_time = datetime.now()
        result = ProcessedTender(tender=tender)
        
        try:
            # STAGE 1: Filter
            print(f"\n{'='*70}")
            print(f"Processing: {tender.title[:50]}...")
            print(f"{'='*70}")
            print("\n[1/3] 🔍 Filtering for relevance...")
            
            result.filter_result = await filter_agent(tender)
            
            print(f"  ✓ Relevant: {result.filter_result.is_relevant}")
            print(f"  ✓ Confidence: {result.filter_result.confidence:.2f}")
            print(f"  ✓ Categories: {[c.value for c in result.filter_result.categories]}")
            
            # Check if we should continue
            if not result.filter_result.is_relevant:
                result.status = "filtered_out"
                print("\n  → ❌ Filtered out (not relevant)")
                return result
            
            if result.filter_result.confidence < self.min_confidence:
                result.status = "filtered_out"
                print(f"\n  → ❌ Filtered out (low confidence < {self.min_confidence})")
                return result
            
            # STAGE 2: Rating
            print(f"\n[2/3] ⭐ Rating opportunity...")
            
            categories = [c.value for c in result.filter_result.categories]
            result.rating_result = await rating_agent(tender, categories)
            
            print(f"  ✓ Overall Score: {result.rating_result.overall_score:.1f}/10")
            print(f"  ✓ Strategic Fit: {result.rating_result.strategic_fit:.1f}/10")
            print(f"  ✓ Win Probability: {result.rating_result.win_probability:.1f}/10")
            
            # Check score threshold
            if result.rating_result.overall_score < self.min_score:
                result.status = "rated_low"
                print(f"\n  → ⚠️  Rated low (score {result.rating_result.overall_score:.1f} < {self.min_score})")
                print(f"  → Skipping document generation")
                return result
            
            # STAGE 3: Document Generation
            print(f"\n[3/3] 📝 Generating bid document...")
            
            result.bid_document = await generator_agent(
                tender,
                categories,
                result.rating_result.strengths
            )
            
            print(f"  ✓ Document generated")
            print(f"  ✓ Executive summary: {len(result.bid_document.executive_summary)} chars")
            
            result.status = "complete"
            print("\n  → ✅ Pipeline complete!")
            
        except Exception as e:
            result.status = "error"
            result.error_message = str(e)
            print(f"\n  → ❌ Error: {e}")
        
        finally:
            result.processing_time = (datetime.now() - start_time).total_seconds()
            print(f"\n  ⏱️  Processing time: {result.processing_time:.2f}s")
        
        return result

print("✓ Orchestrator ready")

## Step 5: Test with High-Quality Tender

Should pass all stages.

In [ ]:
orchestrator = ProcurementOrchestrator()

tender1 = Tender(
    id="ORD-001",
    title="AI-Powered Fraud Detection System",
    description="""Develop machine learning system for real-time fraud detection in 
    financial transactions. Must integrate with existing payment infrastructure, provide 
    dashboard for analysts, and include model training pipeline. 12-month project.""",
    organization="State Financial Crimes Unit",
    deadline="2025-03-01",
    estimated_value="$950K"
)

result1 = await orchestrator.process_tender(tender1)

print("\n" + "="*70)
print("FINAL RESULT SUMMARY")
print("="*70)
print(f"Status: {result1.status}")
print(f"Processing time: {result1.processing_time:.2f}s")
if result1.status == "complete":
    print(f"\n📄 Document preview:")
    print(result1.bid_document.executive_summary[:200] + "...")

## Step 6: Test with Filtered-Out Tender

Should stop at stage 1.

In [ ]:
tender2 = Tender(
    id="ORD-002",
    title="Office Furniture Supply",
    description="Supply 300 office chairs, desks, and filing cabinets. Delivery within 60 days.",
    organization="General Services",
    deadline="2024-11-01",
    estimated_value="$500K"
)

result2 = await orchestrator.process_tender(tender2)

print("\n" + "="*70)
print("FINAL RESULT SUMMARY")
print("="*70)
print(f"Status: {result2.status}")
print(f"Processing time: {result2.processing_time:.2f}s")

## Step 7: Test with Low-Rated Tender

Should pass filter but stop at rating.

In [ ]:
tender3 = Tender(
    id="ORD-003",
    title="National Cloud Infrastructure Modernization Program",
    description="""Massive 5-year, $500M program requiring team of 200+ engineers, 
    proven experience with federal systems, and security clearances. Prime contractor 
    will manage 20+ subcontractors.""",
    organization="Department of Defense",
    deadline="2024-10-15",
    estimated_value="$500M"
)

result3 = await orchestrator.process_tender(tender3)

print("\n" + "="*70)
print("FINAL RESULT SUMMARY")
print("="*70)
print(f"Status: {result3.status}")
print(f"Processing time: {result3.processing_time:.2f}s")
if result3.rating_result:
    print(f"Rating: {result3.rating_result.overall_score:.1f}/10")
    print(f"Recommendation: {result3.rating_result.recommendation[:100]}...")

## Step 8: Batch Processing

Process multiple tenders and analyze results.

In [ ]:
batch_tenders = [
    Tender(
        id="BATCH-1",
        title="Web Portal Modernization",
        description="Modernize citizen services portal with responsive design and accessibility",
        organization="City Digital Services",
        deadline="2025-01-31",
        estimated_value="$650K"
    ),
    Tender(
        id="BATCH-2",
        title="Catering Services Contract",
        description="Daily meal services for government cafeteria",
        organization="Facilities Management",
        deadline="2024-12-01",
        estimated_value="$200K"
    ),
    Tender(
        id="BATCH-3",
        title="Penetration Testing Services",
        description="Quarterly security testing of web applications and APIs",
        organization="IT Security",
        deadline="2024-11-15",
        estimated_value="$180K"
    ),
    Tender(
        id="BATCH-4",
        title="ML-Based Predictive Maintenance",
        description="Build ML models to predict infrastructure failures from IoT sensor data",
        organization="Transportation Authority",
        deadline="2025-02-28",
        estimated_value="$890K"
    ),
]

print("\n" + "="*70)
print(f"BATCH PROCESSING: {len(batch_tenders)} TENDERS")
print("="*70)

results = []
for tender in batch_tenders:
    result = await orchestrator.process_tender(tender)
    results.append(result)

# Summary statistics
print("\n" + "="*70)
print("BATCH SUMMARY")
print("="*70)

status_counts = {}
for result in results:
    status_counts[result.status] = status_counts.get(result.status, 0) + 1

for status, count in status_counts.items():
    print(f"{status}: {count}")

total_time = sum(r.processing_time for r in results)
avg_time = total_time / len(results)

print(f"\nTotal processing time: {total_time:.2f}s")
print(f"Average per tender: {avg_time:.2f}s")

## Step 9: Export Results

In [ ]:
def export_result(result: ProcessedTender) -> dict:
    """Export result to JSON-serializable format"""
    return {
        "tender_id": result.tender.id,
        "tender_title": result.tender.title,
        "status": result.status,
        "processing_time": result.processing_time,
        "filter_result": {
            "is_relevant": result.filter_result.is_relevant if result.filter_result else None,
            "confidence": result.filter_result.confidence if result.filter_result else None,
            "categories": [c.value for c in result.filter_result.categories] if result.filter_result else None
        } if result.filter_result else None,
        "rating_result": {
            "overall_score": result.rating_result.overall_score if result.rating_result else None,
            "recommendation": result.rating_result.recommendation if result.rating_result else None
        } if result.rating_result else None,
        "has_document": result.bid_document is not None
    }

# Export batch results
export_data = [export_result(r) for r in results]
print("\nExported results:")
print(json.dumps(export_data, indent=2))

## 🎉 Congratulations!

You built a complete multi-agent orchestration system!

## What You Learned

1. **Sequential orchestration** - Chain agents with conditional logic
2. **Early termination** - Stop pipeline when conditions aren't met
3. **State management** - Pass data between agents cleanly
4. **Error handling** - Graceful failures with status tracking
5. **Telemetry** - Measure performance and outcomes

## Orchestration Patterns

### Sequential (What We Built)
```
A → B → C
```
Pros: Simple, predictable, easy to debug  
Cons: No parallelism, rigid flow

### Parallel
```
    → B →
A →      → D
    → C →
```
Pros: Faster, independent agents  
Cons: Harder to coordinate, resource intensive

### Graph-Based (LangGraph, etc.)
```
Complex branching and loops
```
Pros: Maximum flexibility  
Cons: Complex, harder to reason about

## Key Design Decisions

| Decision | Rationale |
|----------|----------|
| Sequential execution | Simplest, each stage needs previous results |
| Conditional branching | Don't waste resources on low-quality tenders |
| Configurable thresholds | Business logic separate from agents |
| Status tracking | Clear audit trail |
| Processing time measurement | Optimize bottlenecks |

## Performance Considerations

- **Average processing time:** ~10-30 seconds per tender
- **Bottleneck:** LLM API calls (sequential)
- **Optimization:** Batch processing, caching, faster models

## Next Steps

We have a working system! Now let's make it production-ready:
- Error handling and retries
- Logging and monitoring
- Configuration management
- Deployment strategies

➡️ Continue to `07_production_ready.ipynb`